In [ ]:
import sys
sys.path.append("../")
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
from lora_diffusion import tune_lora_scale, patch_pipe
import open_clip
from PIL import Image

model_id = "/DATA/DATANAS1/zhangyip/models/stable-diffusion-2-1-base"

with torch.no_grad():
    pipe = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")
    img_model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k')
    img_model = img_model.to("cuda")

In [ ]:
patch_pipe(
    pipe,
    "./output_dreambooth/t_disenft_add_no_prior_0.1original_debug/checkpoint/lora_weight_e399_s2400.pt",
    patch_text=True,
    patch_ti=False,
    patch_unet=True,
)
ref_image = preprocess(Image.open("/DATA/DATANAS1/chenhong/diffusion_research/dreambooth_data/backpack/03.jpg")).unsqueeze(0).to("cuda")

In [ ]:
from visualization import joint_visualization
prompt = "a <s1>|<s2> backpack in the ocean"
tune_lora_scale(pipe.unet, 1.0)
image = joint_visualization(pipe, img_model, prompt, ref_image, guidance=7.0, eta=1.0)[0]
image